In [1]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
import plotly.express as px
import pickle
from sklearn.metrics import roc_curve, auc, confusion_matrix
import seaborn.objects as so
import matplotlib.pyplot as plt

In [2]:
# importando dados
df = pd.read_csv('df_esteroids_MB_IN_AR.csv', sep = ';')
df.columns = df.columns.astype(str)
df.columns = ["".join(c if c.isalnum() else "_" for c in str(col)) for col in df.columns]
df


,Classe,__M__,__M_15__,_M_29__,_M_90__,_M_2_90__,_M_3_90_,_M_90_15__,_M_2_90_15__,_M_3_90_15_,...,_M_140_,_M_157_,_M_144_,m_z_103,m_z_129,m_z_143,m_z_169,m_z_244,_m_z_218,m_z_231
0,2,3,3,0,0,0,0,3,3,0,...,0,0,0,0,1,1,2,0,0,0
1,0,3,0,0,0,0,3,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,3,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,2,3,1,0,0,0,0,0,0,0,...,1,0,0,0,1,3,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153,6,2,1,0,3,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
154,6,3,0,0,2,0,0,1,0,0,...,0,0,0,0,0,0,0,0,3,3
155,6,3,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
156,6,1,2,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0


In [3]:
# separando variáveis descritoras
Xi = df.iloc[:,1::]
Xi

,__M__,__M_15__,_M_29__,_M_90__,_M_2_90__,_M_3_90_,_M_90_15__,_M_2_90_15__,_M_3_90_15_,_M_90_29__,...,_M_140_,_M_157_,_M_144_,m_z_103,m_z_129,m_z_143,m_z_169,m_z_244,_m_z_218,m_z_231
0,3,3,0,0,0,0,3,3,0,0,...,0,0,0,0,1,1,2,0,0,0
1,3,0,0,0,0,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,3,1,0,0,0,0,0,0,0,0,...,1,0,0,0,1,3,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153,2,1,0,3,0,0,3,0,0,0,...,0,0,0,0,0,0,0,0,0,0
154,3,0,0,2,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,3,3
155,3,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
156,1,2,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [4]:
# variável target
# separando classe 0(nao-derivado de esteroide) e 1
from collections import Counter

# Supondo que df seja o seu DataFrame

# Inicializando uma nova coluna 'Classe_bin' com valor padrão 0
df['Classe_AAS'] = 0

# Atribuindo valor 1 para o intervalo [91:,:]
df.loc[92:, 'Classe_AAS'] = 1

# Exibindo as contagens após a separação
counter = Counter(df['Classe_AAS'])
y = df['Classe_AAS']
y = np.array(y)
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1])

In [5]:
# balanceando os dados 
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter

# Separando classe 0 (não derivado de esteroide) e 1
y = np.where(df['Classe_AAS'].values == 0, 0, 1)
y = np.array(y)
y = y.ravel()
# Verificando a contagem inicial das classes
print("Contagem inicial das classes:", Counter(y))

# Aplicando UnderSampler usando RandomUnderSampler
rs = RandomUnderSampler(random_state=42)
xi_resampled, y_resampled = rs.fit_resample(Xi, y)

# Verificando a contagem após o UnderSampler
print("Contagem após UnderSampler:", Counter(y_resampled))

Contagem inicial das classes: Counter({0: 92, 1: 66})
Contagem após UnderSampler: Counter({0: 66, 1: 66})


In [8]:
X_train, X_test, y_train, y_test = train_test_split(xi_resampled, y_resampled, test_size=0.33, random_state=29, shuffle=True, stratify=y_resampled)

In [6]:
import pickle
import shap
import xgboost as xgb
import sklearn

In [7]:
with open('XGradientBoosting_anabolizante_model.pkl',  'rb') as f:
    modelo_XGB = pickle.load(f)

In [9]:
pred = modelo_XGB.predict(X_test)

NotFittedError: need to call fit or load_model beforehand

In [26]:
# importando modelos
modelo_XGB = pickle.load(open('XGradientBoosting_anabolizante_model.pkl', 'rb'))
modelo_RF = pickle.load(open('RandomForest_anabolizante_model.pkl', 'rb'))

# Biblioteca
import shap

# Cálculo do SHAP - Definindo explainer com características desejadas
explainer = shap.TreeExplainer(model=modelo_RF)

# Cálculo do SHAP
shap_values_train = explainer.shap_values(x_train, y_train)

AssertionError: Model has no `estimators_`! Have you called `model.fit`?